In [1]:
import random
from Utility import df_ops, music_ops, vmd_ops, io_ops, interpolate, cloud_ops
import pandas as pd
import os 
import pinecone
import pymeshio.common
import numpy as np

class DanceClip:
  def __init__(self, pathToVmd, pathToInterpolatedVmd, startFrame, numberOfFrames):
    self.pathToVmd = pathToVmd
    self.pathToInterpolatedVmd = pathToInterpolatedVmd
    self.startFrame = startFrame
    self.numberOfFrames = numberOfFrames

def queryForSimilarPoses(pinecone, vector, songNamesToExclude):
  index = pinecone.Index(os.getenv('PINECONE_INDEX_NAME'))
  songs = index.query(
    vector=vector,
    filter={
      "animationName": { "$nin": songNamesToExclude }
    },
    top_k=1,
    include_metadata=True
  )
  matchedSongName = songs.matches[0].metadata['animationName']
  frameNumberOfMatchedSong = songs.matches[0].metadata['frameNumber']
  distance = songs.matches[0]['score']
  print("found match: " + matchedSongName + " at frame: " + str(frameNumberOfMatchedSong) + " with distance: " + str(distance))
  return matchedSongName, frameNumberOfMatchedSong

def appendFramesPreservingCenter(df, dfToAppend, buffer=0):
  if df.empty:
    lastPositionOfDf = pymeshio.common.Vector3(0, 0, 0)
  else:
    lastPositionOfDfIdx = df[df['name'] == 'センター']['frame'].idxmax()
    lastPositionOfDf = df.iloc[lastPositionOfDfIdx]['position']

  try:
    firstPositionOfDfToAppendIdx = dfToAppend[dfToAppend['name'] == 'センター']['frame'].idxmin()
    firstPositionOfDfToAppend = dfToAppend.iloc[firstPositionOfDfToAppendIdx]['position']
  except:
    if dfToAppend.empty:
      print("dfToAppend is empty")
    if len(dfToAppend[dfToAppend['name'] == 'センター']) == 0:
      print("No rows with name 'センター' in dfToAppend")
    print("Index to access:", firstPositionOfDfToAppendIdx)
    print("Length of dfToAppend:", len(dfToAppend))
    raise Exception("Error")

  differenceInPos = firstPositionOfDfToAppend - lastPositionOfDf
  differenceInPos.y = 0

  positionBoneNames = ['センター', '右足ＩＫ', '左足ＩＫ']
  dfToAppendWithAdjustedPositions = dfToAppend.copy()
  dfToAppendWithAdjustedPositions['position'] = np.where(dfToAppendWithAdjustedPositions['name'].isin(positionBoneNames), 
                                  dfToAppendWithAdjustedPositions['position'] - differenceInPos,
                                  dfToAppendWithAdjustedPositions['position'])
  return df_ops.appendFrames(df, dfToAppendWithAdjustedPositions, buffer=buffer)

def getVectorFromVectorId(pinecone, songName, framenumber):
  vectorId = songName + "-" + str(int(framenumber))
  print(vectorId)
  index = pinecone.GRPCIndex(os.getenv('PINECONE_INDEX_NAME'))
  fetchVectorId = index.fetch([vectorId])
  return fetchVectorId['vectors'][vectorId]['values']

def generateDance(numberOfFrames):
  INPUT_FOLDER = 'inputData'
  pinecone = cloud_ops.initPinecone()
  sum = 0
  end = numberOfFrames
  newAnimation = pd.DataFrame()
  newInterpolatedAnimation = pd.DataFrame()
  newSongName = random.choice([entry for entry in os.listdir(INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, entry))])
  print("starting with song: " + newSongName)
  pastSongs = [newSongName]

  while sum < end:
    if sum != 0:
      print("querying for similar poses for song: " + prevSongName + " at frame: " + str(startingFrameOfNewFrames))
      poseVector = getVectorFromVectorId(pinecone, prevSongName, startingFrameOfNewFrames + numberOfNewFramesToAdd)
      newSongName, frameNumberOfMatchedSong = queryForSimilarPoses(pinecone, poseVector, pastSongs)
      pastSongs.append(newSongName)

    vmd = [f for f in os.listdir(INPUT_FOLDER + "\\" + newSongName) if f.endswith('.vmd')][0]
    df = vmd_ops.getDfFromVmdFileName(INPUT_FOLDER + "\\" + newSongName + "\\" + vmd)
    numberOfNewFramesToAdd = random.randint(120, 240)
    startingFrameOfNewFrames = frameNumberOfMatchedSong if sum != 0 else random.randint(0, df_ops.getLastFrame(df) - numberOfNewFramesToAdd)
    newAnimation = appendFramesPreservingCenter(newAnimation, df_ops.parseFramesFromDf(df, startingFrameOfNewFrames, startingFrameOfNewFrames + numberOfNewFramesToAdd), bufferFrames)
    if viewInterpolation:
      interpolatedDf = df_ops.loadDfFromFeather(INPUT_FOLDER + "\\" + newSongName + "\\interpolatedMotion.feather")
      newInterpolatedAnimation = df_ops.appendFrames(newInterpolatedAnimation, df_ops.parseFramesFromDf(interpolatedDf, startingFrameOfNewFrames, startingFrameOfNewFrames + numberOfNewFramesToAdd), bufferFrames)
    print("added " + str(numberOfNewFramesToAdd) + " frames. Total frames is now: " + str(df_ops.getLastFrame(newAnimation)))

    sum += numberOfNewFramesToAdd
    prevSongName = newSongName

  vmd_ops.saveDfToVmdFile(newAnimation, savePath + "\\" + saveName + ".vmd")
  if viewInterpolation:
    vmd_ops.saveDfToVmdFile(newInterpolatedAnimation, savePath + "\\" + saveName + "-interpolated-DEBUG.vmd")

def generateVmdAnimationFromDance(animationClips, savePath, saveName, bufferFrames=0, viewInterpolation=False):
  vmd_ops.saveDfToVmdFile(newAnimation, savePath + "\\" + saveName + ".vmd")
  if viewInterpolation:
    vmd_ops.saveDfToVmdFile(newInterpolatedAnimation, savePath + "\\" + saveName + "-interpolated-DEBUG.vmd")

generateDance(1500, 'outputMotions', 'correct-center', bufferFrames=7, viewInterpolation=True)
generateVmdAnimationFromDance()

c:\Users\daryl\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


starting with song: PV002 - World is Mine
added 146 frames. Total frames is now: 153
querying for similar poses for song: PV002 - World is Mine at frame: 4013
PV002 - World is Mine-4159
found match: PV021 - Beware of the Miku Miku Germs at frame: 5000.0 with distance: 0.98961252
added 190 frames. Total frames is now: 350
querying for similar poses for song: PV021 - Beware of the Miku Miku Germs at frame: 5000.0
PV021 - Beware of the Miku Miku Germs-5190
found match: PV626 - Weekender Girl at frame: 1013.0 with distance: 0.988558233
added 224 frames. Total frames is now: 581
querying for similar poses for song: PV626 - Weekender Girl at frame: 1013.0
PV626 - Weekender Girl-1237
found match: PV832 - Hand in Hand at frame: 1262.0 with distance: 0.980824113
added 147 frames. Total frames is now: 735
querying for similar poses for song: PV832 - Hand in Hand at frame: 1262.0
PV832 - Hand in Hand-1409
found match: PV932 - Hand in Hand at frame: 1409.0 with distance: 0.999407589
added 217 fram